## 4_ 파이썬 추천 시스템 패키지 - Surprise

- 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중에 하나인 Surprise 이용


[ 도서 ] [파이썬 머신러닝 완벽 가이드](https://wikibook.co.kr/pymldg-rev/) 예제


---
<< surprise 설치 >>

1. Microsoft Visual Build Tools 2005 이상의 버전 설치 필요

    - https://visualstudio.microsoft.com/ko/downloads/
    - [ Visual Studio 2022 용 도구 ]
    - [ Visual Studio 2022 용 빌드 도구 ] > [ 다운로드 ]
    - vs_buildtools.exe 실행파일
    - [ C++를 사용한 데스크톱 개발 ] > [ 설치 ]
    
2. surprise 설치
    - pip install scikit-surprise
    

## Surprise 추천 알고리즘 

### 1. 추천 알고리즘 클래스

<table>
    <tr><th width='100'>클래스명</th><th>설명</th></tr>
    <tr><td>SVD</td><td>행렬분해를 통한 잠재요인 협업 필터링을 위한 SVD 알고리즘</td></tr>
    <tr><td>KNNBasic</td><td>최근접 이웃 협업 필터링을 위한 KNN 알고리즘</td></tr>
    <tr><td>BaselineOnly</td><td>사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘</td></tr>
</table>

- [ 참조문서 확인 ](http://surprise.readthedocs.io/en/stable/prediction_algorithms_package.html)


### 2. SVD 클래스의 입력 파라미터

<table>
    <tr><th width='100'>파라미터명</th><th>내용</th></tr>
    <tr><td>n_factors</td><td>잠재 요인 K의 개수. 디폴트는 100. 커질수록 정확도는 높아질 수 있지만 과적합 문제가 발생할 수 있음</td></tr>
    <tr><td>n_epochs</td><td>SGD(Stochaslic Gradient Descent) 수행시 반복 횟수. 디폴트는 20. </td></tr>
    <tr><td>biased (bool)</td><td>베이스라인 사용자 편향 적용 여부이며. 디폴트 True </td></tr>
</table>    


### 3. 베이스라인 평점

- 베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수


    - 전체 평균 평점 : 모든 사용자의 아이템에 대한 평점을 평균한 값
    - 사용자 편향 점수 : 사용자별 아이템 평점 평균값 - 전체 평균 평점
    - 아이템 편향 점수 : 아이템별 평점 평균값 - 전체 평균 평점
    
[ex]
   - 어벤져스 영화 평균 평점 : 4.2
   - 모든 사용자의 평균 영화 평점 : 3.5
   - 사용자(홍길동님) 평균 평점 :  3.0
   
   =>  사용자 <홍길동>님의 <어벤저스> 영화 베이스 라인 평점
       
       베이스라인 평점 = 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수
       
                       = 3.5 + ( 3.0 - 3.5 ) + ( 4.2 - 3.5 )
                       = 3.5 + (-0.5) + 0.7
                       = 3.7
                       

In [2]:
# !pip install scikit-surprise

In [3]:
import surprise 

print(surprise.__version__)

1.1.1


## Surprise 를 이용한 추천 시스템 구축

### 1. 제공되는 내장 데이터 세트 이용

In [4]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [5]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0) 

[1] SVD 알고리즘으로 학습 데이타 학습

In [6]:
algo = SVD(random_state=0)
algo.fit(trainset) 

[2] 학습된 추천 알고리즘으로 테스트 데이타 학습

- test() : 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메소드
- predict() : 개별 사용자와 영화에 대한 추천 평점을 반환


+ Prediction 객체의 속성

    - 사용자 아이디(uid) / 영화아이템 아이디(iid) /  실제 평점(r_ui) 
    - Surprise의 추천 예측 평점(est)
    - details 속성에서 'was_impossible': True 이면 예측값을 생성할 수 없는 데이타라는 의미 ( 여기서는 False )

In [7]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [8]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [9]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(120)
iid = str(282)
pred = algo.predict(uid, iid)
print(pred)

user: 120        item: 282        r_ui = None   est = 3.51   {'was_impossible': False}


In [8]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948


### 2. 내장 데이터 세트가 아닌 데이타 이용

In [11]:
import pandas as pd

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv('./dataset/ml-latest-small/ratings_noh.csv', index=False, header=False)


In [13]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('./dataset/ml-latest-small/ratings_noh.csv',reader=reader)

In [14]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

In [16]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### 3. Surprise에서 제공하는 교차 검증과 하이퍼 파라미터 튜닝

In [18]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('./dataset/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8738  0.8788  0.8731  0.8726  0.8759  0.8748  0.0023  
MAE (testset)     0.6726  0.6742  0.6718  0.6702  0.6745  0.6727  0.0016  
Fit time          4.89    5.21    4.85    4.80    4.80    4.91    0.15    
Test time         0.15    0.24    0.21    0.14    0.19    0.19    0.04    


{'test_rmse': array([0.87383233, 0.87881205, 0.87307533, 0.8725828 , 0.8759238 ]),
 'test_mae': array([0.67256527, 0.67423086, 0.67180157, 0.67017743, 0.67450257]),
 'fit_time': (4.8918774127960205,
  5.207037687301636,
  4.850028038024902,
  4.804102659225464,
  4.7951719760894775),
 'test_time': (0.14760255813598633,
  0.23536896705627441,
  0.21342802047729492,
  0.13866782188415527,
  0.19352030754089355)}

In [19]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8787018976504403
{'n_epochs': 20, 'n_factors': 50}


### 4. Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [ ]:
# <오류발생>

'''
# 다음 코드는 train_test_split( )으로 분리되지 않는 데이터 세트에 fit( )을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)
'''

In [21]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성. 
data_folds = DatasetAutoFolds(ratings_file='./dataset/ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함. 
trainset = data_folds.build_full_trainset()


In [22]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)


In [24]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('./dataset/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인. 
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])


사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [25]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)


user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


[ 결과 확인 ]

9번 사용자의 42 번 영화 추천 예측 평점은 : 3.13 

---


In [26]:
def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성. 
    total_movies = movies['movieId'].tolist()
    
    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [27]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions= predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
